# Gather the Data
First step is to gather the data. There is a module defined called RetroSheetDownloader with a clss called DownloadRetroSheetData. Using the class, call the run method that will download the data in .txt format.

In [1]:
from RetroSheetDownloader import DownloadRetroSheetData

# Create an instance of the class and calls the run method. By default, we download the data from the years 2014 to 2019, inclusive 
DownloadRetroSheetData().run()

Unzipping gl2019.zip: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:03<00:00,  1.66log/s]


Next, let's try opening the data with pandas so we can see it in tabular form.

In [12]:
import pandas as pd
import os

# We will be lookin at the 2014 data as an example
retro_sheet_data_gl2014 = pd.read_csv("./datasets/retro_sheet_gls/GL2014.TXT")

# Get all the home games of the Yankees in 2014
nya_gl_2014_home = retro_sheet_data_gl2014.loc[retro_sheet_data_gl2014["Home Team"] == 'NYA']

# Get all the away games of the Yankees in 2014
nya_gl_2014_away = retro_sheet_data_gl2014.loc[retro_sheet_data_gl2014["Visiting Team"] == 'NYA']

# All the Yankees games in 2014 sorted by date
nya_gl_2014 = pd.concat([nya_gl_2014_away, nya_gl_2014_home])
nya_gl_2014.sort_values("Date")

,Date,Number of game,Day,Visiting Team,Visiting Team League,Visiting Team Game Number,Home Team,Home Team League,Home Team Game Number,Visiting Team Score,...,Home Team Player 7 Name,Home Team Player 7 Defensive Position,Home Team Player 8 ID,Home Team Player 8 Name,Home Team Player 8 Defensive Position,Home Team Player 9 ID,Home Team Player 9 Name,Home Team Player 9 Defensive Position,Miscellaneous,Acquisition Information
17,20140401,0,Tue,NYA,AL,1,HOU,AL,1,2,...,Matt Dominguez,5,hoesl001,L.J. Hoes,9,villj001,Jonathan Villar,6,NaN,Y
28,20140402,0,Wed,NYA,AL,2,HOU,AL,2,1,...,Matt Dominguez,5,presa001,Alex Presley,9,villj001,Jonathan Villar,6,NaN,Y
42,20140403,0,Thu,NYA,AL,3,HOU,AL,3,4,...,Matt Dominguez,5,presa001,Alex Presley,9,villj001,Jonathan Villar,6,NaN,Y
56,20140404,0,Fri,NYA,AL,4,TOR,AL,5,7,...,Brett Lawrie,5,goinr001,Ryan Goins,4,diazj004,Jonathan Diaz,6,NaN,Y
71,20140405,0,Sat,NYA,AL,5,TOR,AL,6,0,...,Josh Thole,2,goinr001,Ryan Goins,4,diazj004,Jonathan Diaz,6,NaN,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2362,20140924,0,Wed,BAL,AL,158,NYA,AL,158,9,...,Chris Young,7,richa001,Antoan Richardson,9,ryanb002,Brendan Ryan,6,NaN,Y
2375,20140925,0,Thu,BAL,AL,159,NYA,AL,159,5,...,Stephen Drew,4,suzui001,Ichiro Suzuki,9,pirej001,Jose Pirela,10,NaN,Y
2384,20140926,0,Fri,NYA,AL,160,BOS,AL,160,3,...,Bryce Brentz,7,lavar001,Ryan Lavarnway,3,butld001,Dan Butler,2,NaN,Y
2400,20140927,0,Sat,NYA,AL,161,BOS,AL,161,4,...,Rusney Castillo,8,brenb002,Bryce Brentz,7,vazqc001,Christian Vazquez,2,NaN,Y
